格子上の最短ベクトルの数え上げを行うアルゴリズム<br>
入力： $n$ 次元格子 $L$ の基底のGSO係数 $\mu_{i,j} (1 \leq j < i \leq n)$, 
GSOベクトルの二乗ノルム $B_i = \| \mathbb{b_i^\star} \|^2 (1 \leq i \leq n)$, 
数え上げ上界列 $R_1^2 \leq \cdots \leq R_n^2$<br>
出力：すべての $1 \leq k \leq n$ に対して $\| \pi_k(\mathbb{v}) \|^2 \leq R_{n+1-k}^2$ を満たす格子ベクトル<br>
　　　$\mathbb{v} = \sum_{i=1}^n v_i \mathbb{b_i} \in L$ の係数ベクトル $(v_1, \dots, v_n) \in \mathbb{Z}^n$ ( $v$ が存在する場合)

事前準備

In [8]:
import numpy as np
import math

def input_basis_form():
  print("Enter the dimension of the lattice.")
  print("Enter each basis vector of the lattice")
  n = int(input())
  B = np.array([list(map(int,input().split())) for _ in range(n)])
  return B

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# サブルーチン用
def GSO(B):
  # Bの行数（基底ベクトルの本数）をnとする
  n = len(B)
  # Bと同じサイズのゼロ行列(浮動小数点数成分)を生成
  # ここにGSOの結果を入れていく予定
  B_star = np.zeros_like(B, dtype=float)
  # GSO係数行列のベースを作成
  mu = np.eye(n, dtype=float)

  for i in range(n):
    B_star[i] = B[i]
    for j in range(i):
      mu[i,j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
      B_star[i] -= mu[i,j] * B_star[j]
  # B_star(GSOベクトル行列)とmu(GSO係数行列)を出力
  return B_star, mu

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# メイン処理用
def GSO_main():
  B = input_basis_form()
  return GSO(B)

# GSO係数とGSOベクトルの二乗ノルムを出力するアルゴリズム
def GSO_coeffs_sqnorms(B):
  B_star,mu = GSO(B)
  B_sqnorm = np.array([np.dot(B_star[i],B_star[i]) for i in range(len(B))])
  return B_sqnorm, mu

数え上げ上界列を受け取るアルゴリズムを定義

In [9]:
def input_enum_upperbound_seq():
  print("Enter an enumeration upper bound sequence as R_1^2 R_2^2 ... R_n^2.")
  R = list(map(float, input().split()))
  return R

本題のアルゴリズムを定義（サブルーチン用）

In [12]:
def enum_shortest_vec(B_sqnorm, mu, R):
  n = len(B_sqnorm)
  sigma = np.zeros(shape=(n+1,n), dtype=float)
  r = [i for i in range(n+1)]
  rho = [float(0) for i in range(n+1)]
  v = [0 for i in range(n)]
  c = [0 for i in range(n)]
  w = [0 for i in range(n)]
  v[0] = 1
  last_nonzero = 1
  k=0
  while True:
    rho[k] = rho[k+1] + (v[k] - c[k])**2 * B_sqnorm[k]
    if rho[k] <= R[n-1-k]:
      if k == 0:
        return v
      k -= 1
      r[k] = max(r[k+1], r[k])
      for i in range(r[k+1],k,-1):
        sigma[i,k] = sigma[i+1,k] + mu[i,k] * v[i]
      c[k] = -sigma[k+1,k]
      v[k] = round(c[k])
      w[k] = 1
    else:
      k += 1
      if k == n:
        return "none"
      r[k] = k
      if k >= last_nonzero:
        last_nonzero = k
        v[k] += 1
      else:
        if v[k] > c[k]:
          v[k] -= w[k]
        else:
          v[k] += w[k]
        w[k] += 1

実際に用いてみる

In [24]:
B = np.array([[63, -14, -1, 84, 61],
              [74, -20, 23, -32, -52],
              [93, -46, -19, 0, -63],
              [93, 11, 13, 60, 52],
              [33, -93, 12, 57, -2]])
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("the squared norms of the GSO vectors are")
print(B_sqnorm)
print("GSO coefficient matrix is")
print(mu)

R_5 = B_sqnorm[0] * 0.99
R = [(k+1)/5 * R_5 for k in range(5)]
print(R)
#vol = math.sqrt(np.prod(B_sqnorm))
#upper_bound = math.sqrt(5) ** math.pow(vol, 1/5)
#R = [upper_bound for i in range(5)]
enum_shortest_vec(B_sqnorm, mu, R)

the squared norms of the GSO vectors are
[1.49430000e+04 1.00737428e+04 3.01527354e+03 7.03392732e+02
 3.41094435e-10]
GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [-0.06297263  1.          0.          0.          0.        ]
 [ 0.17928127  1.07305735  1.          0.          0.        ]
 [ 0.93046912  0.31890545 -0.43777128  1.          0.        ]
 [ 0.53770996  0.33393597  0.72786965 -2.94334071  1.        ]]
[np.float64(2958.714), np.float64(5917.428), np.float64(8876.142), np.float64(11834.856), np.float64(14793.57)]


[0, 1, 0, 0, 0]